In [ ]:
import torch
import pandas as pd

from ark_nlp.model.tm.sbert import Module
from ark_nlp.model.tm.sbert import ModuleConfig
from ark_nlp.model.tm.sbert import Dataset
from ark_nlp.model.tm.sbert import Task
from ark_nlp.model.tm.sbert import get_default_model_optimizer
from ark_nlp.model.tm.sbert import Tokenizer
from ark_nlp.factory.utils.seed import set_seed

In [ ]:
set_seed(42)

In [ ]:
# 目录地址
# 数据集下载地址：http://icrc.hitsz.edu.cn/info/1037/1146.htm

train_data_path = '../data/source_datasets/LCQMC/train.txt'
dev_data_path = '../data/source_datasets/LCQMC/dev.txt'

In [ ]:
model_name = 'nghuyong/ernie-1.0'

### 一、数据读入与处理

#### 1. 数据读入

In [ ]:
train_data_df = pd.read_csv(train_data_path, sep='\t', names=['text_a', 'text_b', 'label'])
train_data_df = (train_data_df
                 .loc[:,['text_a', 'text_b', 'label']])

dev_data_df = pd.read_csv(dev_data_path, sep='\t', names=['text_a', 'text_b', 'label'])
dev_data_df = (dev_data_df
                 .loc[:,['text_a', 'text_b', 'label']])

In [ ]:
sbert_train_dataset = Dataset(train_data_df)
sbert_dev_dataset = Dataset(dev_data_df)

#### 2. 词典创建和生成分词器

In [ ]:
# 加载分词器
tokenizer = Tokenizer(vocab=model_name, max_seq_len=64)

#### 3. ID化

In [ ]:
sbert_train_dataset.convert_to_ids(tokenizer)
sbert_dev_dataset.convert_to_ids(tokenizer)

<br>

### 二、模型构建

#### 1. 模型参数设置

In [ ]:
bert_config = ModuleConfig.from_pretrained(
    model_name,
    num_labels=2
)

In [ ]:
torch.cuda.empty_cache()

#### 2. 模型创建

In [ ]:
dl_module = Module.from_pretrained(
    model_name,
    config=bert_config,
    dropout=0.3
)

<br>

### 三、任务构建

#### 1. 任务参数和必要部件设定

In [ ]:
# 设置运行次数
num_epoches = 10
batch_size = 64

In [ ]:
param_optimizer = list(dl_module.named_parameters())
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]     

#### 2. 任务创建

In [ ]:
model = Task(dl_module, 'adamw', 'ce', cuda_device=0)

#### 3. 训练

In [ ]:
model.fit(
    sbert_train_dataset,
    sbert_dev_dataset,
    lr=1e-5,
    epochs=1,
    batch_size=64,
    params=optimizer_grouped_parameters
)

<br>

### 四、模型验证与保存

#### 1. 模型验证

In [ ]:
from ark_nlp.model.tm.unsupervised_simcse import Predictor

sbert_predictor_instance = Predictor(model.module, tokenizer, sbert_dev_dataset.cat2id)

In [ ]:
sbert_predictor_instance.predict_one_sample(['感冒', '恐惧'])

In [ ]:
sbert_predictor_instance.predict_one_sample(['感冒', '恐惧'], return_proba=True)

In [ ]:
sbert_predictor_instance.predict_one_sample(['感冒', '恐惧'], threshold=None)

#### 2. Batch模型验证

In [ ]:
test_data_path = '../data/source_datasets/LCQMC/test.txt'

In [ ]:
test_data_df = pd.read_csv(test_data_path, sep='\t', names=['text_a', 'text_b', 'label'])
test_data_df = (test_data_df
                 .loc[:,['text_a', 'text_b', 'label']])

In [ ]:
sbert_test_dataset = Dataset(test_data_df, categories=sbert_train_dataset.categories, is_test=True)
sbert_test_dataset.convert_to_ids(tokenizer)

In [ ]:
predict_label = sbert_predictor_instance.predict_batch(sbert_test_dataset)

#### 3. 多样本验证

In [ ]:
test_data_path = '../data/source_datasets/LCQMC/test.txt'
test_data_df = pd.read_csv(test_data_path, sep='\t', names=['text_a', 'text_b', 'label'])
test_data_df = (test_data_df
                 .loc[:,['text_a', 'text_b', 'label']])

record_ = []
for _text_a, _text_b in zip(test_data_df['text_a'], test_data_df['text_b']):
    record_.append([_text_a, _text_b, sbert_predictor_instance.predict_one_sample([_text_a, _text_b])])